## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-06-39-28 +0000,nypost,"Trump to host Edan Alexander, American who spe...",https://nypost.com/2025/10/07/us-news/trump-to...
1,2025-10-07-06-36-51 +0000,nyt,Flight Delays Begin as Air Traffic Staffing Sh...,https://www.nytimes.com/2025/10/06/us/politics...
2,2025-10-07-06-33-56 +0000,bbc,Starmer urges students not to protest on 7 Oct...,https://www.bbc.com/news/articles/c1wgx5v90vyo...
3,2025-10-07-06-31-54 +0000,bbc,What are the key sticking points in Trump's Ga...,https://www.bbc.com/news/articles/c930v44dg2ro...
4,2025-10-07-06-28-00 +0000,wsj,German Factory Orders Unexpectedly Fall Again ...,https://www.wsj.com/economy/german-factory-ord...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2448/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,52
256,new,25
280,government,16
281,shutdown,15
266,deal,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
68,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,126
41,2025-10-07-01-27-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,120
306,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...,116
89,2025-10-06-22-14-52 +0000,latimes,Trump slams judge he picked as court tests lim...,https://www.latimes.com/california/story/2025-...,114
146,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...,114


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
68,126,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
137,83,2025-10-06-20-29-42 +0000,nyt,"Paramount Buys Bari Weiss’s Free Press, Starti...",https://www.nytimes.com/2025/10/06/business/me...
146,62,2025-10-06-20-04-32 +0000,nypost,Trump appealing ruling against Portland Nation...,https://nypost.com/2025/10/06/us-news/trump-ap...
38,44,2025-10-07-01-58-41 +0000,nyt,Israel and Hamas Meet in Egypt to Discuss Gaza...,https://www.nytimes.com/2025/10/06/world/middl...
162,42,2025-10-06-19-11-11 +0000,bbc,US Supreme Court rejects Ghislaine Maxwell app...,https://www.bbc.com/news/articles/cn83e6q7lq4o...
13,39,2025-10-07-04-59-56 +0000,nypost,Ex-LA deputy mayor of public safety sentenced ...,https://nypost.com/2025/10/07/us-news/ex-la-de...
19,38,2025-10-07-04-21-31 +0000,nypost,Local station’s chief meteorologist allegedly ...,https://nypost.com/2025/10/07/us-news/louisian...
40,37,2025-10-07-01-30-25 +0000,nypost,"Young girls, 12 and 13, killed in horrific NYC...",https://nypost.com/2025/10/06/us-news/young-gi...
270,34,2025-10-06-12-00-00 +0000,cbc,"'No more losing,' Poilievre tells Carney in sc...",https://www.cbc.ca/news/politics/poilievre-sca...
287,33,2025-10-06-10-00-00 +0000,nypost,"These Americans died fighting in Ukraine, now ...",https://nypost.com/2025/10/06/world-news/paren...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
